In [70]:
import pickle

import numpy as np
import torch
import torch.nn.functional as F

from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from torch import nn, optim

from skorch.dataset import CVSplit
from skorch.callbacks import EpochScoring
from skorch.classifier import NeuralNetClassifier
from skorch.net import NeuralNet

In [27]:
X, y = make_classification(n_samples=1000, n_features=20,
                           n_informative=10, n_classes=3,
                           random_state=123)

X = X.astype(np.float32)
y = y.astype(np.int64)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, stratify=y, train_size=0.8)
print(f"Type X: {X_train.dtype}")
print(f"Type y: {y_train.dtype}")


Type X: float32
Type y: int64


In [59]:
class NetClassifier(nn.Module):
    
    def __init__(self, num_units=64):
        super(NetClassifier, self).__init__()
        self.num_units = num_units
        
        self.linear = nn.Linear(20, self.num_units)
        self.linear2 = nn.Linear(self.num_units, 32)
        self.linear3 = nn.Linear(32, 3)
        self.dropout = nn.Dropout(p=0.2)
    
    def forward(self, x):
        x = F.relu(self.linear(x))
        x = self.dropout(x)
        x = F.relu(self.linear2(x))
        x = F.softmax(self.linear3(x), dim=1)
        return x

    
acc_train = EpochScoring(scoring=accuracy_score, lower_is_better=False, on_train=True)
acc_valid = EpochScoring(scoring=accuracy_score, lower_is_better=False, on_train=False)
    
net = NeuralNetClassifier(
    module = NetClassifier,
    #module__num_units = 54,
    criterion=nn.NLLLoss,
    optimizer=optim.Adam,
    #optimizer__lr=1e-3,
    lr=1e-3,
    max_epochs=25,
    device='cuda',
    train_split=CVSplit(cv=0.15, stratified=True),
    callbacks=[acc_train]
)

net.fit(X_train, y_train)
y_class = net.predict(X_test)
y_prob = net.predict_proba(X_test)


  epoch    accuracy_score    train_loss    valid_acc    valid_loss     dur
-------  ----------------  ------------  -----------  ------------  ------
      1            0.3103        1.1075       0.3833        1.0779  0.0411
      2            0.4809        1.0574       0.5333        1.0481  0.0361
      3            0.5588        1.0288       0.5667        1.0235  0.0356
      4            0.5529        1.0006       0.6000        0.9996  0.0351
      5            0.6132        0.9743       0.5667        0.9746  0.0341
      6            0.6088        0.9456       0.5583        0.9480  0.0351
      7            0.6191        0.9174       0.5417        0.9219  0.0321
      8            0.6412        0.8785       0.5500        0.8956  0.0361
      9            0.6485        0.8507       0.5833        0.8698  0.0326
     10            0.6735        0.8218       0.6167        0.8450  0.0336
     11            0.6794        0.7908       0.6250        0.8225  0.0346
     12            0.7059

In [36]:
y_class

array([0, 0, 1, 2, 1, 1, 0, 2, 0, 2, 0, 0, 2, 0, 1, 2, 0, 1, 1, 1, 2, 0,
       2, 2, 1, 1, 0, 2, 2, 2, 0, 1, 2, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1,
       2, 2, 1, 2, 0, 0, 1, 2, 1, 0, 2, 0, 2, 1, 0, 1, 1, 2, 2, 0, 2, 1,
       0, 1, 0, 0, 0, 1, 0, 2, 1, 0, 0, 0, 1, 0, 2, 1, 2, 0, 0, 0, 1, 0,
       1, 2, 2, 0, 1, 0, 2, 0, 0, 1, 0, 2, 2, 2, 1, 0, 0, 2, 1, 1, 0, 1,
       0, 0, 1, 2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 2, 2,
       0, 1, 0, 1, 2, 2, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 0, 2, 0, 2, 0, 0, 2, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 2, 1, 2, 1, 0, 2, 2, 0, 0, 0, 0, 1, 2, 0, 1, 2, 2,
       2, 1], dtype=int64)

In [37]:
y_prob

array([[8.91190708e-01, 5.14783897e-02, 5.73309213e-02],
       [7.14790165e-01, 1.87224895e-01, 9.79849398e-02],
       [1.29795969e-01, 5.23964942e-01, 3.46239090e-01],
       [2.42265776e-01, 3.26228857e-01, 4.31505442e-01],
       [1.89428613e-01, 7.90573239e-01, 1.99981071e-02],
       [2.36641631e-01, 5.70246994e-01, 1.93111360e-01],
       [7.59905636e-01, 2.05117185e-02, 2.19582722e-01],
       [1.92739591e-01, 2.07736343e-03, 8.05182993e-01],
       [7.02628613e-01, 1.33170620e-01, 1.64200738e-01],
       [3.58929396e-01, 3.43587361e-02, 6.06711864e-01],
       [6.05406523e-01, 7.82932267e-02, 3.16300303e-01],
       [6.38577461e-01, 1.32435396e-01, 2.28987187e-01],
       [3.38282287e-02, 2.22261343e-03, 9.63949084e-01],
       [8.48609984e-01, 9.11165550e-02, 6.02733940e-02],
       [5.54128364e-02, 8.85331213e-01, 5.92559539e-02],
       [3.87716182e-02, 1.19336978e-01, 8.41891468e-01],
       [5.08691430e-01, 4.55509275e-02, 4.45757687e-01],
       [5.54309860e-02, 7.53975

## Sklearn Pipeline

In [50]:
pipe = Pipeline([
    ('net', net)
])

pipe.fit(X_train, y_train)
y_class = pipe.predict(X_test)
y_prob = pipe.predict_proba(X_test)


Re-initializing module because the following parameters were re-set: num_units.
Re-initializing optimizer because the following parameters were re-set: lr.
  epoch    accuracy_score    train_loss    valid_acc    valid_loss     dur
-------  ----------------  ------------  -----------  ------------  ------
      1            0.3794        1.0921       0.5000        1.0707  0.0582
      2            0.4559        1.0609       0.5750        1.0428  0.0401
      3            0.5235        1.0251       0.5917        1.0151  0.0356
      4            0.5529        0.9966       0.6167        0.9860  0.0343
      5            0.5574        0.9623       0.6500        0.9553  0.0396
      6            0.5882        0.9358       0.6417        0.9238  0.0346
      7            0.5926        0.9066       0.6750        0.8918  0.0321
      8            0.6088        0.8755       0.6833        0.8607  0.0356
      9            0.6397        0.8483       0.6833        0.8310  0.0381
     10            

## GridSearchCV

In [51]:
net.prefixes_

['module',
 'iterator_train',
 'iterator_valid',
 'optimizer',
 'criterion',
 'callbacks',
 'dataset']

In [62]:
pipe = Pipeline([
    ('net', net)
])

grid = {
    'net__module__num_units': [50, 60]
}

gs = GridSearchCV(
    estimator=pipe, 
    param_grid=grid,
    scoring='accuracy',
    cv=3,
    verbose=2
)

gs.fit(X_train, y_train)
predictions = gs.predict(X_test)


Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] net__module__num_units=50 .......................................
Re-initializing module because the following parameters were re-set: num_units.
Re-initializing optimizer.
Re-initializing module because the following parameters were re-set: num_units.
Re-initializing optimizer.
  epoch    accuracy_score    train_loss    valid_acc    valid_loss     dur
-------  ----------------  ------------  -----------  ------------  ------
      1            0.3451        1.1052       0.3125        1.0943  0.0317
      2            0.3982        1.0775       0.4250        1.0732  0.0251
      3            0.4690        1.0531       0.4625        1.0542  0.0221
      4            0.5221        1.0383       0.5125        1.0360  0.0256
      5            0.5177        1.0239       0.5125        1.0187  0.0241

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



      6            0.5310        1.0083       0.5250        1.0022  0.0276
      7            0.5686        0.9847       0.5500        0.9865  0.0231
      8            0.5597        0.9761       0.5875        0.9707  0.0251
      9            0.5885        0.9507       0.6000        0.9563  0.0261
     10            0.6195        0.9243       0.6250        0.9432  0.0251
     11            0.6283        0.9185       0.6250        0.9317  0.0231
     12            0.6438        0.8856       0.6375        0.9219  0.0241
     13            0.6173        0.8762       0.6375        0.9127  0.0231
     14            0.6482        0.8548       0.6375        0.9025  0.0221
     15            0.6350        0.8450       0.6625        0.8915  0.0226
     16            0.6460        0.8260       0.6375        0.8809  0.0261
     17            0.6681        0.7979       0.6375        0.8726  0.0206
     18            0.6681        0.7785       0.6250        0.8657  0.0271
     19            0.681

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


      6            0.5629        0.9761       0.5625        0.9683  0.0241
      7            0.6115        0.9422       0.5750        0.9507  0.0226
      8            0.5982        0.9270       0.6000        0.9330  0.0261
      9            0.6093        0.9099       0.6250        0.9161  0.0226
     10            0.6358        0.8868       0.6375        0.8993  0.0216
     11            0.6093        0.8696       0.6375        0.8828  0.0216
     12            0.6645        0.8553       0.6375        0.8665  0.0256
     13            0.6313        0.8365       0.6375        0.8514  0.0281
     14            0.6534        0.8102       0.6375        0.8369  0.0246
     15            0.6667        0.7860       0.6500        0.8232  0.0231
     16            0.6865        0.7687       0.6375        0.8098  0.0261
     17            0.6667        0.7546       0.6500        0.7976  0.0226
     18            0.6954        0.7447       0.6750        0.7867  0.0256
     19            0.7108

[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    4.8s finished



      5            0.5324        0.9688       0.5417        1.0006  0.0321
      6            0.5765        0.9388       0.5417        0.9783  0.0371
      7            0.5809        0.9148       0.5500        0.9555  0.0321
      8            0.5971        0.8950       0.5750        0.9303  0.0346
      9            0.6147        0.8672       0.5917        0.9027  0.0321
     10            0.6088        0.8532       0.5750        0.8759  0.0316
     11            0.6471        0.8238       0.5917        0.8511  0.0321
     12            0.6456        0.8002       0.6167        0.8284  0.0366
     13            0.6529        0.7944       0.6500        0.8067  0.0321
     14            0.6794        0.7652       0.6750        0.7864  0.0316
     15            0.6647        0.7513       0.6833        0.7672  0.0316
     16            0.6956        0.7384       0.6833        0.7510  0.0331
     17            0.6735        0.7171       0.6833        0.7376  0.0311
     18            0.692

In [68]:
confusion_matrix(y_test, predictions)

array([[47,  6, 18],
       [ 6, 51,  6],
       [14,  9, 43]], dtype=int64)

## Saving model

In [71]:
# Save model gs
with open('model.pkl', 'wb') as f:
    pickle.dump(gs, f)


c:\users\se.vi.dmitriev\python-virtual-environments\pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type NetClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
c:\users\se.vi.dmitriev\python-virtual-environments\pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
c:\users\se.vi.dmitriev\python-virtual-environments\pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Dropout. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
c:\users\se.vi.dmitriev\python-virtual-environments\pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code 

In [72]:
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)
